In [4]:
import numpy as np

np.set_printoptions(formatter={'int':lambda x:hex(int(x))})

# Sarwate's algo

In [78]:
# sawates
#data = list(range(16)) # data[0] is first
data = [0 for i in range(16)] # data[0] is first

table = np.zeros(256, dtype=int)
poly = 0xedb88320 # this is the bit reversed polynomial 0x4C11DB7
for n in range(256):
    c = n
    for k in range(8):
        c = poly ^ (c >> 1) if c & 1 else c >> 1
    table[n] = c


# do the crc
crc = 0xFFFFFFFF # setting the initial crc to 0xFFFFFFFF is equivilent to complimenting first 32 bits,

print('prev_crc, idx, tout, new_crc')                 
for x in data:
    print(hex(crc), (crc ^ x) & 0xFF, hex(table[(crc ^ x) & 0xFF]), hex((crc >> 8) ^ table[(crc ^ x) & 0xFF]))
    crc = (crc >> 8) ^ table[(crc ^ x) & 0xFF]
    

print('CRC=', hex(crc ^ 0xFFFFFFFF))

prev_crc, idx, tout, new_crc
0xffffffff 255 0x2d02ef8d 0x2dfd1072
0x2dfd1072 114 0xbe0b1010 0xbe26ed00
0xbe26ed00 0 0x0 0xbe26ed
0xbe26ed 237 0xdebb9ec5 0xdebb20e3
0xdebb20e3 227 0x3903b3c2 0x39dd08e2
0x39dd08e2 226 0x4e048354 0x4e3d5e5c
0x4e3d5e5c 92 0x62dd1ddf 0x62932081
0x62932081 129 0x9abfb3b6 0x9add2096
0x9add2096 150 0x196c3671 0x19f6eb51
0x19f6eb51 81 0x1c6c6162 0x1c759789
0x1c759789 137 0x94643b84 0x94784e13
0x94784e13 19 0x84be41de 0x842a3990
0x842a3990 144 0xf00f9344 0xf08bb97d
0xf08bb97d 125 0x2eb40d81 0x2e448638
0x2e448638 56 0x2802b89e 0x282cfc18
0x282cfc18 24 0x136c9856 0x1344b4aa
CRC= 0xecbb4b55


# Slicing CRC algorithm

References:
- https://web.archive.org/web/20120722193753/http://download.intel.com:80/technology/comms/perfnet/download/slicing-by-8.pdf
- https://create.stephan-brumme.com/crc32/#slicing-by-8-overview  

In [29]:
# generate table of all 8-bit input combinations

poly = 0x04C11DB7
bit_reverse_poly = True

poly = int('{:032b}'.format(poly)[::-1], 2) if bit_reverse_poly else poly

def byte_lookup():
    table = []
    for d in range(256):
        t = d
        for i in range(8):
            t = (t >> 1) ^ ((t & 0x01) * poly)
        table.append(t)
    return table

In [79]:
crcTables = np.zeros((8, 256), dtype=int)
crcTables[0] = byte_lookup()

for j in range(1, 8):
    for i in range(256):
        crcTables[j][i] = (crcTables[j-1][i] >> 8) ^ crcTables[0][crcTables[j-1][i] & 0xFF]

with open('../hdl/crc_tables.mem', 'w') as f:
    f.writelines([l + '\n' for l in [' '.join([f'{ti:08x}' for ti in t]) for t in crcTables]])

crcTables

array([[0x0, 0x77073096, 0xee0e612c, ..., 0xc30c8ea1, 0x5a05df1b,
        0x2d02ef8d],
       [0x0, 0x191b3141, 0x32366282, ..., 0xa1129ff0, 0x8a3fcc33,
        0x9324fd72],
       [0x0, 0x1c26a37, 0x384d46e, ..., 0xbd1ce083, 0xbf5a5eda,
        0xbe9834ed],
       ...,
       [0x0, 0xcb5cd3a5, 0x4dc8a10b, ..., 0xa90c0ac7, 0x2f987869,
        0xe4c4abcc],
       [0x0, 0xa6770bb4, 0x979f1129, ..., 0x5a95b19, 0x34414184,
        0x92364a30],
       [0x0, 0xccaa009e, 0x4225077d, ..., 0x646e019b, 0xeae10678,
        0x264b06e6]])

In [31]:
def chunker(seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))

slice_byte = lambda x, i : (x >> (8 * i) ) & 0xFF # Returns ith least sig byte

In [52]:
data = list(range(16))
crc = 0 ^ 0xFFFFFFFF

for id in chunker(data, 8):

    # data[0] is first (low) byte
    # xor the first 4 (lowest 4) bytes with the prev crc
    lookup = np.array([d  ^ slice_byte(crc, i) if i < 4 else d for i, d in enumerate(id)])

    print(lookup)

    # use these values to index the lookup tables
    tout = np.array([crcTables[::-1][i][l] for i, l in enumerate(lookup)]) # not table[0] is for last (highest) byte

    print(tout)

    xor_tout = 0

    for t in tout:
        xor_tout = xor_tout ^ t
    
    crc = xor_tout
    print(hex(xor_tout))

print('CRC=', hex(xor_tout ^ 0xFFFFFFFF))

[0xff 0xfe 0xfd 0xfc 0x4 0x5 0x6 0x7]
[0x264b06e6 0x34414184 0xa90c0ac7 0x8dc4bd5c 0x8f629757 0x6cbc2eb
 0x565aa786 0x9e6495a3]
0x77559760
[0x68 0x9e 0x5f 0x7c 0xc 0xd 0xe 0xf]
[0x30ebb0e 0xbff70d61 0xfdd8ba22 0x3c220dce 0x4ad6bfb8 0x8d89353
 0x9e832d8e 0x90bf1d91]
0x31311d77
CRC= 0xcecee288


# Reduced lookup table method

Algo from https://www.semanticscholar.org/paper/Design-of-high-speed-CRC-algorithm-for-ethernet-on-Bajarangbali-Anand/43806dc970b0c36ecc7a2912eecbc0efaf119a90#related-papers

Using method described in http://repository.sustech.edu:8080/bitstream/handle/123456789/22823/Design%20of%20an%20Efficient%20...%20.pdf?sequence=1  pg 20

Reduced table lookup

In [89]:
lut1 = [ 0xB8BC6765, 0xAA09C88B, 0x8F629757, 0xC5B428EF, 0x5019579F, 0xA032AF3E, 0x9B14583D, 0xED59B63B, 0x1C26A37, 0x384D46E, 0x709A8DC, 0xE1351B8, 0x1C26A370, 0x384D46E0, 0x709A8DC0, 0xE1351B80, 0x191B3141, 0x32366282, 0x646CC504, 0xC8D98A08, 0x4AC21251, 0x958424A2, 0xF0794F05, 0x3B83984B, 0x77073096, 0xEE0E612C, 0x76DC419, 0xEDB8832, 0x1DB71064, 0x3B6E20C8, 0x76DC4190, 0xEDB88320, 
]

lut2 = [ 0xCCAA009E, 0x4225077D, 0x844A0EFA, 0xD3E51BB5, 0x7CBB312B, 0xF9766256, 0x299DC2ED, 0x533B85DA, 0xA6770BB4, 0x979F1129, 0xF44F2413, 0x33EF4E67, 0x67DE9CCE, 0xCFBD399C, 0x440B7579, 0x8816EAF2, 0xCB5CD3A5, 0x4DC8A10B, 0x9B914216, 0xEC53826D, 0x3D6029B, 0x7AC0536, 0xF580A6C, 0x1EB014D8, 0x3D6029B0, 0x7AC05360, 0xF580A6C0, 0x30704BC1, 0x60E09782, 0xC1C12F04, 0x58F35849, 0xB1E6B092, 
]

lut3 = [ 0x9BA54C6F, 0xEC3B9E9F, 0x3063B7F, 0x60C76FE, 0xC18EDFC, 0x1831DBF8, 0x3063B7F0, 0x60C76FE0, 0xC18EDFC0, 0x586CB9C1, 0xB0D97382, 0xBAC3E145, 0xAEF6C4CB, 0x869C8FD7, 0xD64819EF, 0x77E1359F, 0xEFC26B3E, 0x4F5D03D, 0x9EBA07A, 0x13D740F4, 0x27AE81E8, 0x4F5D03D0, 0x9EBA07A0, 0xE6050901, 0x177B1443, 0x2EF62886, 0x5DEC510C, 0xBBD8A218, 0xACC04271, 0x82F182A3, 0xDE920307, 0x6655004F,
]

lut4 = [ 0xAE689191, 0x87A02563, 0xD4314C87, 0x73139F4F, 0xE6273E9E, 0x173F7B7D, 0x2E7EF6FA, 0x5CFDEDF4, 0xB9FBDBE8, 0xA886B191, 0x8A7C6563, 0xCF89CC87, 0x44629F4F, 0x88C53E9E, 0xCAFB7B7D, 0x4E87F0BB, 0x9D0FE176, 0xE16EC4AD, 0x19AC8F1B, 0x33591E36, 0x66B23C6C, 0xCD6478D8, 0x41B9F7F1, 0x8373EFE2, 0xDD96D985, 0x605CB54B, 0xC0B96A96, 0x5A03D36D, 0xB407A6DA, 0xB37E4BF5, 0xBD8D91AB, 0xA06A2517, 
]

luts = np.array([lut1, lut2, lut3, lut4])

In [90]:
crc = 0
data = [0, 0, 0, 0] # 4 x 32 bits
t = np.zeros(4, dtype=int)
t[0] = data[0]
t[1] = data[1]
t[2] = data[2]
t[3] = data[3] ^ (crc ^ 0xFFFFFFFF)


for i in range(32):
    for j in range(4):
        lookup = (t[j] >> i) & 0x01 == 1
        if lookup:
            crc = crc ^ luts[j][i]

        print(f'{i} {j} {t[j]:08x} {lookup} {crc:08x}')
        
crc = crc ^ 0xFFFFFFFF
print(f"Final CRC = {crc:08x}")

0 0 00000000 False 00000000
0 1 00000000 False 00000000
0 2 00000000 False 00000000
0 3 ffffffff True ae689191
1 0 00000000 False ae689191
1 1 00000000 False ae689191
1 2 00000000 False ae689191
1 3 ffffffff True 29c8b4f2
2 0 00000000 False 29c8b4f2
2 1 00000000 False 29c8b4f2
2 2 00000000 False 29c8b4f2
2 3 ffffffff True fdf9f875
3 0 00000000 False fdf9f875
3 1 00000000 False fdf9f875
3 2 00000000 False fdf9f875
3 3 ffffffff True 8eea673a
4 0 00000000 False 8eea673a
4 1 00000000 False 8eea673a
4 2 00000000 False 8eea673a
4 3 ffffffff True 68cd59a4
5 0 00000000 False 68cd59a4
5 1 00000000 False 68cd59a4
5 2 00000000 False 68cd59a4
5 3 ffffffff True 7ff222d9
6 0 00000000 False 7ff222d9
6 1 00000000 False 7ff222d9
6 2 00000000 False 7ff222d9
6 3 ffffffff True 518cd423
7 0 00000000 False 518cd423
7 1 00000000 False 518cd423
7 2 00000000 False 518cd423
7 3 ffffffff True 0d7139d7
8 0 00000000 False 0d7139d7
8 1 00000000 False 0d7139d7
8 2 00000000 False 0d7139d7
8 3 ffffffff True b48ae23f
9

Reduced table lookup - one xor per non-zero bit. For 64 bit input - 32 xors on crit path, not ideal. ~0.2ns per LUT.

64 bit version:

In [117]:
crc = 0
for j in range(2):
    for i in range(32):
        crc = crc ^ luts[2][i]

hex(crc)

'0x0'

In [120]:
crc = 0
data = [0, 0] # 2 x 32 bits
t = np.zeros(4, dtype=int)
t[0] = data[0]
t[1] = data[1] ^ (crc ^ 0xFFFFFFFF)

for j in range(2):
    for i in range(32):
    
        lookup = (t[j] >> i) & 0x01 == 1
        if lookup:
            crc = crc ^ luts[j][i]

        print(f'{i} {j} {t[j]:08x} {lookup} {crc:08x}')
            
    
crc = crc ^ 0xFFFFFFFF
print(f"Final CRC = {crc:08x}") # for 64'h0 should be = 6522df69

0 0 00000000 False 00000000
1 0 00000000 False 00000000
2 0 00000000 False 00000000
3 0 00000000 False 00000000
4 0 00000000 False 00000000
5 0 00000000 False 00000000
6 0 00000000 False 00000000
7 0 00000000 False 00000000
8 0 00000000 False 00000000
9 0 00000000 False 00000000
10 0 00000000 False 00000000
11 0 00000000 False 00000000
12 0 00000000 False 00000000
13 0 00000000 False 00000000
14 0 00000000 False 00000000
15 0 00000000 False 00000000
16 0 00000000 False 00000000
17 0 00000000 False 00000000
18 0 00000000 False 00000000
19 0 00000000 False 00000000
20 0 00000000 False 00000000
21 0 00000000 False 00000000
22 0 00000000 False 00000000
23 0 00000000 False 00000000
24 0 00000000 False 00000000
25 0 00000000 False 00000000
26 0 00000000 False 00000000
27 0 00000000 False 00000000
28 0 00000000 False 00000000
29 0 00000000 False 00000000
30 0 00000000 False 00000000
31 0 00000000 False 00000000
0 1 ffffffff True ccaa009e
1 1 ffffffff True 8e8f07e3
2 1 ffffffff True 0ac50919
3